In [ ]:
# DO NOT CONTAINERISE
# parametrization - v95
# Run specific - fill in text to preset, keep empty to prompt
param_radar = "herwijnen"  # denhelder,herwijnen
param_start_date = (
    "2019-12-31T23:00+00:00"  # %Y%m%dT%H:%M+TZ; 2019-12-31T23:00+00:00
)
param_end_date = (
    "2020-01-01T01:00+00:00"  # %Y%m%dT%H:%M+TZ; 2020-01-01T01:00+00:00
)
param_concurrency = 5
param_interval = 60
# This is to control uploads / cleaning
# Move results to S3?
param_upload_results = "True"
# Remove input after processing KNMI format to ODIM format
param_clean_knmi_input = "True"
# Should we remove the final Polar Volumes after producing a VP and or RBC
param_clean_pvol_output = "True"
#
param_clean_vp_output = "True"
# User specific, not neccesarily run specific
param_minio_user_pvol_output_prefix = ""  # "<username>/pvol"
param_minio_user_vp_output_prefix = ""  # "<username>/vp"
# Secret management
param_secret_file_path = "/home/jovyan/secrets.yaml"
param_key_knmi_api = "secret_knmi_api_key"
param_key_minio_access_key = "secret_minio_access_key"
param_key_minio_secret_key = "secret_minio_secret_key"
param_key_minio_endpoint = "secret_minio_endpoint"

# Param
### User specific, not neccesarily run specific.
#### Update: Perhaps some of these userinfo should be hardcoded parameters. I mean
#### Not something we'd enter every single time but should be set at setup time.
param_minio_user_pvol_output_prefix = "bwijers1@gmail.com/pvol"
param_minio_user_vp_output_prefix = "bwijers1@gmail.com/vp"
param_minio_user_ppi_output_prefix = "bwijers1@gmail.com/ppi"
param_minio_user_regvpts_output_prefix = "bwijers1@gmail.com/regvpts"

#### Visualization parameters
param_dtype = "pvol"  # pvol | vp
param_country = "NL"  # only NL
param_radar = "DHL"  # DHL | HRW | DBL
param_year = "2023"  # as string, YYYY
param_month = "12"  # as string, mm
param_day = "31"  # as string, dd

# Perhaps param prefix works better
param_prefix = "NL/DHL/2023/12/31"

# parameters
param_elevation = 3  #
param_param = "VRADH"  # I know...VRADH, DBZH, TH, WRADH, RHOHV, DBZ...
param_imtype = "ppi"  # Likely only type so far. PVOL -> PPI, VP -> VPTS. Future there would be more.

In [ ]:
# DO NOT CONTAINERISE
# configuration - v95
import os
import pathlib

conf_minio_user_bucket_name = "naa-vre-user-data"  # the user bucket name
conf_minio_public_bucket_name = "naa-vre-public"  # the public bucket name
conf_minio_public_root_prefix = (
    "vl-vol2bird"  # the prefix to use in the public bucket
)
conf_minio_public_conf_prefix = (
    "vl-vol2bird/conf"  # Path to the public configuration file
)
conf_minio_public_conf_radar_db_object_name = (
    "vl-vol2bird/conf/OPERA_RADARS_DB.json"
)

### Directories
conf_local_root = "/tmp/data"
conf_local_knmi = "/tmp/data/knmi"
conf_local_odim = "/tmp/data/odim"
conf_local_vp = "/tmp/data/vp"
conf_local_conf = "/tmp/data/conf"
conf_local_radar_db = "/tmp/data/conf/OPERA_RADARS_DB.json"
conf_local_visualization_input = "/tmp/data/visualizations/input"
conf_local_visualization_output = "/tmp/data/visualizatons/output"

# radar configuration for the KNMI api
# Rewritten in a long format without page breaks. This is to prevent
# the code analyzer to yield an error.
# datasetName, datasetVersion, api_url, radar code (odim)
conf_herwijnen = [
    "radar_volume_full_herwijnen",
    1.0,
    "https://api.dataplatform.knmi.nl/open-data/v1/datasets/radar_volume_full_herwijnen/versions/1.0/files",
    "NL/HRW",
]
conf_denhelder = [
    "radar_volume_full_denhelder",
    2.0,
    "https://api.dataplatform.knmi.nl/open-data/v1/datasets/radar_volume_denhelder/versions/2.0/files",
    "NL/DHL",
]
conf_radars = {"herwijnen": conf_herwijnen, "denhelder": conf_denhelder}

In [ ]:
# DO NOT CONTAINERISE - v60
# Set initial resource in minio, the conf in common
debug = False
from minio import Minio, S3Error
import os
import pathlib

minioClient = Minio(
    endpoint=param_minio_endpoint,
    access_key=param_minio_access_key,
    secret_key=param_minio_secret_key,
    secure=True,
)
# Stat object to see if it is there. We could also just try a try except on fget
# However, canonically this seems better to stat
# Cast to str as stat object doesnt like posix
try:
    radar_db_stat = minioClient.stat_object(
        bucket_name=pathlib.Path(conf_minio_public_bucket_name).as_posix(),
        object_name=pathlib.Path(
            conf_minio_public_conf_radar_db_object_name
        ).as_posix(),
    )
    print(
        f"Reference file found [{pathlib.Path(conf_minio_public_bucket_name.as_posix())}]/{pathlib.Path(conf_minio_public_conf_radar_db_object_name).as_posix()}"
    )
    print(radar_db_stat)

except S3Error as e:
    print(
        f"Failed to find reference file [{pathlib.Path(conf_minio_public_bucket_name).as_posix()}]/{pathlib.Path(conf_minio_public_conf_radar_db_object_name).as_posix()}"
    )
    if debug:
        print(f"{e=}")

    file_stat = os.stat("/home/jovyan/data/conf/OPERA_RADARS_DB.json")
    with open(
        "/home/jovyan/data/conf/OPERA_RADARS_DB.json", mode="rb"
    ) as file_data:
        put_result = minioClient.put_object(
            bucket_name=pathlib.Path(conf_minio_public_bucket_name).as_posix(),
            object_name=pathlib.Path(
                conf_minio_public_conf_radar_db_object_name
            ).as_posix(),
            data=file_data,
            length=file_stat.st_size,
        )
    print(f"{put_result=}")
    # check put result if we indeed uploaded succesfully
    # if we dont try except we will lazily evaluate the result, as fail likely yields hard crash
    print("Succesfully uploaded reference file")

In [ ]:
# initializer
import pathlib
from secrets_provider import SecretsProvider

# Retrieve secrets
secret_provider = SecretsProvider(param_secret_file_path)
secret_minio_access_key = secret_provider.get(param_key_minio_access_key)
secret_minio_secret_key = secret_provider.get(param_key_minio_secret_key)
secret_minio_endpoint = secret_provider.get(param_key_minio_endpoint)

# Make directories on shared (local) storage
for local_dir in [
    conf_local_root,
    conf_local_knmi,
    conf_local_odim,
    conf_local_vp,
    conf_local_conf,
]:
    local_dir = pathlib.Path(local_dir)
    if not local_dir.exists():
        local_dir.mkdir(parents=True, exist_ok=True)
# Reference files
if not pathlib.Path(conf_local_radar_db).exists():
    from minio import Minio, S3Error

    minioClient = Minio(
        endpoint=secret_minio_endpoint,
        access_key=secret_minio_access_key,
        secret_key=secret_minio_secret_key,
        secure=True,
    )
    print(f"{conf_local_radar_db} not found, downloading")
    minioClient.fget_object(
        bucket_name=conf_minio_public_bucket_name,
        object_name=conf_minio_public_conf_radar_db_object_name,
        file_path=conf_local_radar_db,
    )

# Now produce a variable which acts as a marker for the workflow manager
# We can then drag a line from the configuration / initializer
# and time the start of the rest of the workflow
# If you decide to make different sets of configurations, you can store them
# and decide per workflow which config to attach
init_complete = "Yes"  # Cant sent bool
print("Finished initialization")

In [ ]:
# DO NOT CONTAINERIZE Get KNMI api key
from secrets_provider import SecretsProvider

param_home_directory_path = "/home/jovyan"
param_secrets_file_name = "secrets.yaml"
param_KMNI_key_name = "KNMI_OPEN_DATA_API_KEY"
secret_file_location: str = (
    param_home_directory_path + "/" + param_secrets_file_name
)
secret_KNMI_API_key: str = SecretsProvider(secret_file_location).get_secret(
    param_KMNI_key_name
)

In [ ]:
# list-knmi-files
"""
consume dummy var from config to signal workflow start
There is something dodgy going on with how
strings are being passed around.
The string "Yes" is being sent as '"Yes"'
So, to prevent extra quotes being introduced
we eval init_complete first before
we test if it contains "Yes"
"""
# Libraries
import requests
from secrets_provider import SecretsProvider

# Retrieve the API key from secrets
secret_knmi_api_key = SecretsProvider(param_secret_file_path).get_secret(
    param_key_knmi_api
)

# Strip any extra quotes
init_complete = init_complete.replace("'", "")
init_complete = init_complete.replace('"', "")
if init_complete == "Yes":
    print("Workflow configuration succesfull")
else:
    print("Workflow configuration was not complete, exitting")
    import sys

    sys.exit(1)

# Notes:
# Timestamps in iso8601
# 2020-01-01T00:00+00:00

# configure
start_ts = param_start_date
end_ts = param_end_date
datasetName, datasetVersion, api_url, _ = conf_radars.get(param_radar)
params = {
    "datasetName": datasetName,
    "datasetVersion": datasetVersion,
    "maxKeys": 10,
    "sorting": "asc",
    "orderBy": "created",
    "begin": start_ts,
    "end": end_ts,
}
# Request a response from the KNMI severs
# Try the next page tokens
dataset_files = []
while True:
    list_files_response = requests.get(
        url=api_url,
        headers={"Authorization": secret_knmi_api_key},
        params=params,
    )
    list_files = list_files_response.json()
    dset_files = list_files.get("files")
    dset_files = [list(dset_file.values()) for dset_file in dset_files]
    dataset_files += dset_files
    nextPageToken = list_files.get("nextPageToken")
    if not nextPageToken:
        break
    else:
        params.update({"nextPageToken": nextPageToken})

# KNMI outputs per 5 minutes, per 15 is less of a heavy hit on downloads and processing
# Quick and dirty way to only keep the 15 minute measurements.
# Check API if we can filter for this on their end. If not fine
filtered_list = []
interval_list = list(range(0, 60, param_interval))
for dataset_file in dataset_files:
    minute = int(dataset_file[0].split("_")[-1].split(".")[0][-2:])
    if minute in interval_list:
        filtered_list.append(dataset_file)

dataset_files = filtered_list
print(f"Found {len(dataset_files)} files")
print(dataset_files)

In [ ]:
# KNMI-to-ODIM-converter
"""
notes: 
Need to add this such that it can upload the PVOL From this stage
Need to add option such that this can remove the PVOL files from this stage. 
Warning, with the removal of PVOL on this stage auto-bricks the VP / RBC gen
We can introduce a flag check where RBC and VP check if PVOL 'needed' to be removed
If that flag is met - abort, there 'shouldnt' be any INPUT files then. 
"""
import subprocess
import pathlib
import h5py
import json
import sys
import shutil
from secrets_provider import SecretsProvider

# from typing import List, Object
import math

# Retrieve secrets
secret_provider = SecretsProvider(param_secret_file_path)
secret_minio_access_key = secret_provider.get(param_key_minio_access_key)
secret_minio_secret_key = secret_provider.get(param_key_minio_secret_key)
secret_minio_endpoint = secret_provider.get(param_key_minio_endpoint)


def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ("yes", "true", "t", "y", "1"):
        return True
    elif v.lower() in ("no", "false", "f", "n", "0"):
        return False
    else:
        raise Exception


class FileTranslatorFileTypeError(LookupError):
    """raise this when there's a filetype mismatch derived from h5 file"""


def load_radar_db(radar_db_path):
    """Load and return the radar database

    Output dict sample (wmo code is used as key):
    {
        11038: {'number': '1209', 'country': 'Austria', 'countryid': 'LOWM41', 'oldcountryid': 'OS41', 'wmocode': '11038', 'odimcode': 'atrau', 'location': 'Wien/Schwechat', 'status': '1', 'latitude': '48.074', 'longitude': '16.536', 'heightofstation': ' ', 'band': 'C', 'doppler': 'Y', 'polarization': 'D', 'maxrange': '224', 'startyear': '1978', 'heightantenna': '224', 'diametrantenna': ' ', 'beam': ' ', 'gain': ' ', 'frequency': '5.625', 'single_rrr': 'Y', 'composite_rrr': 'Y', 'wrwp': 'Y'},
        11052: {'number': '1210', 'country': 'Austria', 'countryid': 'LOWM43', 'oldcountryid': 'OS43', 'wmocode': '11052', 'odimcode': 'atfel', 'location': 'Salzburg/Feldkirchen', 'status': '1', 'latitude': '48.065', 'longitude': '13.062', 'heightofstation': ' ', 'band': 'C', 'doppler': 'Y', 'polarization': 'D', 'maxrange': '224', 'startyear': '1992', 'heightantenna': '581', 'diametrantenna': ' ', 'beam': ' ', 'gain': ' ', 'frequency': '5.6', 'single_rrr': 'Y', 'composite_rrr': ' ', 'wrwp': ' '},
        ...
    }
    """
    with open(radar_db_path, mode="r") as f:
        radar_db_json = json.load(f)
    radar_db = {}
    # Reorder list to a usable dict with sub dicts which we can search with wmo codes
    for radar_dict in radar_db_json:
        try:
            wmo_code = int(radar_dict.get("wmocode"))
            radar_db.update({wmo_code: radar_dict})
        except Exception:  # Happens when there is for ex. no wmo code.
            pass
    return radar_db


def translate_wmo_odim(radar_db, wmo_code):
    """ """
    if not isinstance(wmo_code, int):
        raise ValueError("Expecting a wmo_code [int]")
    else:
        pass
    odim_code = (
        radar_db.get(wmo_code).get("odimcode").upper().strip()
    )  # Apparently, people sometimes forget to remove whitespace..
    return odim_code


def extract_wmo_code(in_path):
    with h5py.File(in_path, mode="r") as f:
        # DWD Specific
        # Main attributes
        what = f["what"].attrs
        # Source block
        source = what.get("source")
        source = source.decode("utf-8")
        # Determine if we are dealing with a WMO code or with an ODIM code set
        # Example from Germany where source block is set as WMO
        # what/source: "WMO:10103"
        # Example from The Netherlands where source block is set as a combination of ODIM and various codes
        # what/source: RAD:NL52,NOD:nlhrw,PLC:Herwijnen
        source_list = source.split(sep=",")
    wmo_code = [string for string in source_list if "WMO" in string]
    # Determine if we had exactly one WMO hit
    if len(wmo_code) == 1:
        wmo_code = wmo_code[0]
        wmo_code = wmo_code.replace("WMO:", "")
    # No wmo code found, most likeley dealing with a dutch radar
    elif len(wmo_code) == 0:
        rad_str = [string for string in source_list if "RAD" in string]

        if len(rad_str) == 1:
            rad_str = rad_str[0]
        else:
            print(
                "Something went wrong with determining the rad_str and it wasnt WMO either, exitting"
            )
            sys.exit(1)
        # Split the rad_str
        rad_str_split = rad_str.split(":")
        # [0] = RAD, [1] = rad code
        rad_code = rad_str_split[1]

        rad_codes = {"NL52": "6356", "NL51": "6234", "NL50": "6260"}

        wmo_code = rad_codes.get(rad_code)
    return int(wmo_code)


def translate_knmi_filename(in_path_h5):
    wmo_code = extract_wmo_code(in_path_h5)
    odim_code = translate_wmo_odim(radar_db, wmo_code)
    with h5py.File(in_path_h5, mode="r") as f:
        what = f["what"].attrs
        # Date block
        date = what.get("date")
        date = date.decode("utf-8")
        # Time block
        time = what.get("time")
        # time = f['dataset1/what'].attrs['endtime']
        time = time.decode("utf-8")
        hh = time[:2]
        mm = time[2:4]
        ss = time[4:]
        time = time[:-2]  # Do not include seconds
        # File type
        filetype = what.get("object")
        filetype = filetype.decode("utf-8")
        if filetype != "PVOL":
            raise FileTranslatorFileTypeError("File type was NOT pvol")
    name = [
        odim_code,
        filetype.lower(),
        date + "T" + time,
        str(wmo_code) + ".h5",
    ]
    ibed_fname = "_".join(name)
    return ibed_fname


def knmi_to_odim(in_fpath, out_fpath):
    """
    Converter usage:
    Usage: KNMI_vol_h5_to_ODIM_h5 ODIM_file.h5 KNMI_input_file.h5

    Returns out_fpath and returncode
    """
    converter = "/opt/radar/vol2bird/bin/./KNMI_vol_h5_to_ODIM_h5"
    command = [converter, out_fpath, in_fpath]
    proc = subprocess.run(command, stderr=subprocess.PIPE)
    output = proc.stderr.decode("utf-8")
    returncode = int(proc.returncode)
    return (out_fpath, returncode, output)


print(f"{knmi_pvol_paths=}")
odim_pvol_paths = []
radar_db = load_radar_db(conf_local_radar_db)
for knmi_path in knmi_pvol_paths:
    out_path_pvol_odim = pathlib.Path(knmi_path.replace("knmi", "odim"))
    print(f"{knmi_path=}")
    print(f"{out_path_pvol_odim=}")
    if not out_path_pvol_odim.parent.exists():
        out_path_pvol_odim.parent.mkdir(parents=True, exist_ok=False)
    converter_results = knmi_to_odim(
        in_fpath=str(knmi_path), out_fpath=str(out_path_pvol_odim)
    )
    print(f"{converter_results=}")
    if param_clean_knmi_input:
        pathlib.Path(knmi_path).unlink()
        if not any(pathlib.Path(knmi_path).parent.iterdir()):
            pathlib.Path(knmi_path).parent.rmdir()
    # Determine name for our convention
    ibed_pvol_name = translate_knmi_filename(in_path_h5=out_path_pvol_odim)
    out_path_pvol_odim_tce = pathlib.Path(out_path_pvol_odim).parent.joinpath(
        ibed_pvol_name
    )
    shutil.move(src=out_path_pvol_odim, dst=out_path_pvol_odim_tce)
    odim_pvol_paths.append(out_path_pvol_odim_tce)

print(f"{odim_pvol_paths=}")
if str2bool(param_upload_results):
    # Minio version
    from minio import Minio

    minioClient = Minio(
        endpoint=secret_minio_endpoint,
        access_key=secret_minio_access_key,
        secret_key=secret_minio_secret_key,
        secure=True,
    )
    print(f"Uploading results to {param_minio_user_pvol_output_prefix}")
    for odim_pvol_path in odim_pvol_paths:
        odim_pvol_path = pathlib.Path(odim_pvol_path)
        local_pvol_storage = pathlib.Path(conf_local_odim)
        relative_path = odim_pvol_path.relative_to(local_pvol_storage)
        remote_odim_pvol_path = Path(
            param_minio_user_pvol_output_prefix
        ).joinpath(relative_path)
        # check if this exists
        exists = False
        try:
            _ = minioClient.stat_object(
                bucket=conf_minio_user_bucket_name,
                prefix=remote_odim_pvol_path.as_posix(),
            )
            exists = True
        except:
            pass
        if not exists:
            print(f"Uploading {odim_pvol_path} to {remote_odim_pvol_path}")
            with open(odim_pvol_path, mode="rb") as file_data:
                file_stat = os.stat(odim_pvol_path)
                minioClient.put_object(
                    bucket_name=conf_minio_user_bucket_name,
                    object_name=remote_odim_pvol_path.as_posix(),
                    data=file_data,
                    length=file_stat.st_size,
                )
        else:
            print(f"{remote_odim_pvol_path} exists, skipping ")
    print("Finished uploading results")
# cast to string to not break json serializer
odim_pvol_paths = [path.as_posix() for path in odim_pvol_paths]

In [ ]:
# PVOL-VP-converter
import pandas as pd
import re
import pathlib
from secrets_provider import SecretsProvider

# Retrieve secrets
secret_provider = SecretsProvider(param_secret_file_path)
secret_minio_access_key = secret_provider.get(param_key_minio_access_key)
secret_minio_secret_key = secret_provider.get(param_key_minio_secret_key)
secret_minio_endpoint = secret_provider.get(param_key_minio_endpoint)


def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ("yes", "true", "t", "y", "1"):
        return True
    elif v.lower() in ("no", "false", "f", "n", "0"):
        return False
    else:
        raise Exception


def load_radar_db(radar_db_path):
    """Load and return the radar database
    Output dict sample (wmo code is used as key):
    {
        11038: {'number': '1209', 'country': 'Austria', 'countryid': 'LOWM41', 'oldcountryid': 'OS41', 'wmocode': '11038', 'odimcode': 'atrau', 'location': 'Wien/Schwechat', 'status': '1', 'latitude': '48.074', 'longitude': '16.536', 'heightofstation': ' ', 'band': 'C', 'doppler': 'Y', 'polarization': 'D', 'maxrange': '224', 'startyear': '1978', 'heightantenna': '224', 'diametrantenna': ' ', 'beam': ' ', 'gain': ' ', 'frequency': '5.625', 'single_rrr': 'Y', 'composite_rrr': 'Y', 'wrwp': 'Y'},
        11052: {'number': '1210', 'country': 'Austria', 'countryid': 'LOWM43', 'oldcountryid': 'OS43', 'wmocode': '11052', 'odimcode': 'atfel', 'location': 'Salzburg/Feldkirchen', 'status': '1', 'latitude': '48.065', 'longitude': '13.062', 'heightofstation': ' ', 'band': 'C', 'doppler': 'Y', 'polarization': 'D', 'maxrange': '224', 'startyear': '1992', 'heightantenna': '581', 'diametrantenna': ' ', 'beam': ' ', 'gain': ' ', 'frequency': '5.6', 'single_rrr': 'Y', 'composite_rrr': ' ', 'wrwp': ' '},
        ...
    }
    """
    with open(radar_db_path, mode="r") as f:
        radar_db_json = json.load(f)
    radar_db = {}
    # Reorder list to a usable dict with sub dicts which we can search with wmo codes
    for radar_dict in radar_db_json:
        try:
            wmo_code = int(radar_dict.get("wmocode"))
            radar_db.update({wmo_code: radar_dict})
        except Exception:  # Happens when there is for ex. no wmo code.
            pass
    return radar_db


def translate_wmo_odim(radar_db, wmo_code):
    """"""
    # class FileTranslatorFileTypeError(LookupError):
    #    '''raise this when there's a filetype mismatch derived from h5 file'''
    if not isinstance(wmo_code, int):
        raise ValueError("Expecting a wmo_code [int]")
    else:
        pass
    odim_code = (
        radar_db.get(wmo_code).get("odimcode").upper().strip()
    )  # Apparently, people sometimes forget to remove whitespace..
    return odim_code


def extract_wmo_code(in_path):
    with h5py.File(in_path, "r") as f:
        # DWD Specific
        # Main attributes
        what = f["what"].attrs
        # Source block
        source = what.get("source")
        source = source.decode("utf-8")
        # Determine if we are dealing with a WMO code or with an ODIM code set
        # Example from Germany where source block is set as WMO
        # what/source: "WMO:10103"
        # Example from The Netherlands where source block is set as a combination of ODIM and various codes
        # what/source: RAD:NL52,NOD:nlhrw,PLC:Herwijnen
        source_list = source.split(sep=",")
    wmo_code = [string for string in source_list if "WMO" in string]
    # Determine if we had exactly one WMO hit
    if len(wmo_code) == 1:
        wmo_code = wmo_code[0]
        wmo_code = wmo_code.replace("WMO:", "")
    # No wmo code found, most likeley dealing with a dutch radar
    elif len(wmo_code) == 0:
        rad_str = [string for string in source_list if "RAD" in string]
        if len(rad_str) == 1:
            rad_str = rad_str[0]
        else:
            print(
                "Something went wrong with determining the rad_str and it wasnt WMO either, exiting"
            )
            sys.exit(1)
        # Split the rad_str
        rad_str_split = rad_str.split(":")
        # [0] = RAD, [1] = rad code
        rad_code = rad_str_split[1]
        rad_codes = {"NL52": "6356", "NL51": "6234", "NL50": "6260"}
        wmo_code = rad_codes.get(rad_code)
    return int(wmo_code)


def vol2bird(
    in_file,
    out_dir,
    radar_db,
    add_version=True,
    add_sector=False,
    overwrite=False,
):
    # Construct output file
    date_regex = "([0-9]{8})"
    if add_version == True:
        version = "v0-3-20"
        suffix = pathlib.Path(in_file).suffix
        in_file_name = pathlib.Path(in_file).name
        in_file_stem = pathlib.Path(in_file_name).stem
        #
        out_file_name = in_file_stem.replace("pvol", "vp")
        out_file_name = "_".join([out_file_name, version]) + suffix
        # odim = odim_code(out_file_name)
        wmo = extract_wmo_code(in_file)
        odim = translate_wmo_odim(radar_db, wmo)
        datetime = pd.to_datetime(re.search(date_regex, out_file_name)[0])
        ibed_path = "/".join(
            [
                odim[:2],
                odim[2:],
                str(datetime.year),
                str(datetime.month).zfill(2),
                str(datetime.day).zfill(2),
            ]
        )
        # check if we need to make this dir
        out_file = "/".join([out_dir, ibed_path, out_file_name])
        out_file_dir = pathlib.Path(out_file).parent
        if not out_file_dir.exists():
            out_file_dir.mkdir(parents=True)

    process = False
    if not overwrite:
        if not pathlib.Path(out_file).exists():
            process = True
            print(f"Not processing, overwrite is set to {overwrite}")
    else:
        process = True

    if process:
        command = ["vol2bird", in_file, out_file]
        result = subprocess.run(
            command, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT
        )
    return [in_file, out_file]


vertical_profile_paths = []
radar_db = load_radar_db(conf_local_radar_db)
# cast back to pathlib after deserializing
odim_pvol_paths = [pathlib.Path(path) for path in odim_pvol_paths]
for odim_pvol_path in odim_pvol_paths:
    pvol_path, vp_path = vol2bird(
        odim_pvol_path, conf_local_vp, radar_db, overwrite=False
    )
    vertical_profile_paths.append(vp_path)
print(vertical_profile_paths)

if str2bool(param_clean_pvol_output):
    print("Removing PVOL output from local storage")
    for pvol_path in odim_pvol_paths:
        pathlib.Path(pvol_path).unlink()
        if not any(pathlib.Path(pvol_path).parent.iterdir()):
            pathlib.Path(pvol_path).parent.rmdir()

if str2bool(param_upload_results):
    # Minio version
    from minio import Minio

    minioClient = Minio(
        endpoint=secret_minio_endpoint,
        access_key=secret_minio_access_key,
        secret_key=secret_minio_secret_key,
        secure=True,
    )
    print(f"Uploading results to {param_minio_user_vp_output_prefix}")
    for vp_path in vertical_profile_paths:
        vp_path = pathlib.Path(vp_path)
        local_vp_storage = pathlib.Path(conf_local_vp)
        relative_path = vp_path.relative_to(local_vp_storage)
        remote_vp_path = pathlib.Path(
            param_minio_user_vp_output_prefix
        ).joinpath(relative_path)
        # check if this exists
        exists = False
        try:
            _ = minioClient.stat_object(
                bucket=conf_minio_user_bucket_name,
                prefix=remote_vp_path.as_posix(),
            )
            exists = True
        except:
            pass
        if not exists:
            print(f"Uploading {vp_path} to {remote_vp_path}")
            with open(vp_path, mode="rb") as file_data:
                file_stat = os.stat(vp_path)
                minioClient.put_object(
                    bucket_name=conf_minio_user_bucket_name,
                    object_name=remote_vp_path.as_posix(),
                    data=file_data,
                    length=file_stat.st_size,
                )
        else:
            print(f"{remote_vp_path} exists, skipping ")
    print("Finished uploading results")
if str2bool(param_clean_vp_output):
    print("Removing VP output from local storage")
    for vp_path in vertical_profile_paths:
        pathlib.Path(vp_path).unlink()
        if not any(pathlib.Path(vp_path).parent.iterdir()):
            pathlib.Path(vp_path).parent.rmdir()

In [ ]:
# Retrieve PVOL 
from minio import Minio
import sys
import pathlib

#
minioClient = Minio(
    endpoint=secret_minio_endpoint,
    access_key=secret_minio_access_key,
    secret_key=secret_minio_secret_key,
    secure=True,
)

# dtype = "pvol"
# country = "NL"
# radar = "DHL"
# year = "2023"
# month = "12"
# day = "31"
recursive = True

if param_dtype.lower() in ["pvol", "polar volume", "polarvolume"]:
    search_prefix = f"{param_minio_user_pvol_output_prefix}/{param_country}/{param_radar}/{param_year}/{param_month}/{param_day}"
elif param_dtype.lower() in ["vp", "vertical profile", "verticalprofile"]:
    search_prefix = f"{param_minio_user_vp_output_prefix}/{param_country}/{param_radar}/{param_year}/{param_month}/{param_day}"
else:
    print(f"{param_dtype} not understood")
    sys.exit(1)
print(f"{search_prefix=}")
# To be implemented:
# The below works, but we can use this for the filtering from parameters at some point.
# This shoud be done after the demo version.
# start_after_prefix=f'{param_minio_user_pvol_output_prefix}/{country}/{radar}/{year}/{month}/{day}/{country}{radar}_{dtype}_{year}{month}{day}T2200_6234.h5'
# print(f"{start_after_prefix=}")
# objects = minioClient.list_objects(bucket_name=conf_minio_user_bucket_name,
#                                   prefix=search_prefix,
#                                   recursive=recursive,
#                                   start_after=start_after_prefix
#                                  )
objects = minioClient.list_objects(
    bucket_name=conf_minio_user_bucket_name,
    prefix=search_prefix,
    recursive=recursive,
)
local_file_paths = []
for obj in objects:
    obj_path = pathlib.Path(obj._object_name)
    local_file_path = f"{conf_local_visualization_input}/{obj_path.name}"
    local_file_paths.append(local_file_path)
    print(f"Downloading {obj._object_name} to {local_file_path}")
    minioClient.fget_object(
        bucket_name=obj._bucket_name,
        object_name=obj._object_name,
        file_path=local_file_path,
    )
    local_file_paths.append(local_file_path)
print("Finished")

In [ ]:
# Retrieve PVOL 
from minio import Minio
import sys
import pathlib

#
minioClient = Minio(
    endpoint=secret_minio_endpoint,
    access_key=secret_minio_access_key,
    secret_key=secret_minio_secret_key,
    secure=True,
)

# dtype = "pvol"
# country = "NL"
# radar = "DHL"
# year = "2023"
# month = "12"
# day = "31"
recursive = True

if param_dtype.lower() in ["pvol", "polar volume", "polarvolume"]:
    search_prefix = f"{param_minio_user_pvol_output_prefix}/{param_country}/{param_radar}/{param_year}/{param_month}/{param_day}"
elif param_dtype.lower() in ["vp", "vertical profile", "verticalprofile"]:
    search_prefix = f"{param_minio_user_vp_output_prefix}/{param_country}/{param_radar}/{param_year}/{param_month}/{param_day}"
else:
    print(f"{param_dtype} not understood")
    sys.exit(1)
print(f"{search_prefix=}")
# To be implemented:
# The below works, but we can use this for the filtering from parameters at some point.
# This shoud be done after the demo version.
# start_after_prefix=f'{param_minio_user_pvol_output_prefix}/{country}/{radar}/{year}/{month}/{day}/{country}{radar}_{dtype}_{year}{month}{day}T2200_6234.h5'
# print(f"{start_after_prefix=}")
# objects = minioClient.list_objects(bucket_name=conf_minio_user_bucket_name,
#                                   prefix=search_prefix,
#                                   recursive=recursive,
#                                   start_after=start_after_prefix
#                                  )
objects = minioClient.list_objects(
    bucket_name=conf_minio_user_bucket_name,
    prefix=search_prefix,
    recursive=recursive,
)
local_file_paths = []
for obj in objects:
    obj_path = pathlib.Path(obj._object_name)
    local_file_path = f"{conf_local_visualization_input}/{obj_path.name}"
    local_file_paths.append(local_file_path)
    print(f"Downloading {obj._object_name} to {local_file_path}")
    minioClient.fget_object(
        bucket_name=obj._bucket_name,
        object_name=obj._object_name,
        file_path=local_file_path,
    )
    local_file_paths.append(local_file_path)
print("Finished")

In [ ]:
# Upload Images
# Perhaps this block consumes a list of files coming from the R block. 
# testing
from minio import Minio
import sys
import pathlib
# 
minioClient = Minio(
    endpoint=secret_minio_endpoint,
    access_key=secret_minio_access_key,
    secret_key=secret_minio_secret_key,
    secure=True,
)

# local_path = pathlib.Path("/tmp/data/visualizations/output/NLDHL_ppi_3_20231231T2215_6234.png")
for local_path in local_image_paths:
    if param_imtype == "ppi":
        object_name = f"{param_minio_user_ppi_output_prefix}/{param_prefix}/{local_path.name}"
    elif param_imtype == "regvpts":
        object_name = f"{param_minio_user_revpts_output_prefix}/{param_prefix}/{local_path.name}"
    elif param_imtype == "vpts"
        object_name = f"{param_minio_user_vpts_output_prefix}/{param_prefix}/{local_path.name}"
    else:
        print(f"{param_imtype=} not supported")
        sys.exit(1)

    obj_write_result = minioClient.fput_object(bucket_name=conf_minio_user_bucket_name,
                            object_name=object_name,
                            file_path=local_path.as_posix()
                           )